 # F710で操作しながらデータ取り
 別途 02, interactive_regressionを実行

### カメラの初期化

In [1]:
from jetcam.csi_camera import CSICamera
# from jetcam.usb_camera import USBCamera

camera = CSICamera(width=224, height=224)
# camera = USBCamera(width=224, height=224)

camera.running = True

test


### racecarクラスのインスタンスを生成します。

In [2]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()
#デフォルトの上書き必須！
car.steering_gain   = 0.45
car.steering_offset = -0.1
car.throttle_gain   = 0.65
car.throttle_offset = 0.2 

### データセットのタスクの定義

In [3]:
import torchvision.transforms as transforms
from xy_dataset import XYDataset

TASK = 'road_following'

CATEGORIES = ['apex']

DATASETS = ['A','donkey']
#DATASETS = ['A','donkey']

TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

datasets = {}
for name in DATASETS:
    datasets[name] = XYDataset(TASK + '_' + name, CATEGORIES, TRANSFORMS, random_hflip=True)

### データ収集(コントローラ操作用）

In [ ]:
camera.running = True
#~ここはデフォルトのデータ取り~~~~~~~
import cv2
import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from jupyter_clickable_image_widget import ClickableImageWidget

# initialize active dataset
dataset = datasets[DATASETS[0]]

# unobserve all callbacks from camera in case we are running this cell for second time
camera.unobserve_all()

# create image preview
camera_widget = ClickableImageWidget(width=camera.width, height=camera.height)
snapshot_widget = ipywidgets.Image(width=camera.width, height=camera.height)
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# create widgets
dataset_widget = ipywidgets.Dropdown(options=DATASETS, description='dataset')
category_widget = ipywidgets.Dropdown(options=dataset.categories, description='category')
count_widget = ipywidgets.IntText(description='count')

# manually update counts at initialization
count_widget.value = dataset.get_count(category_widget.value)

# sets the active dataset
def set_dataset(change):
    global dataset
    dataset = datasets[change['new']]
    count_widget.value = dataset.get_count(category_widget.value)
dataset_widget.observe(set_dataset, names='value')

# update counts when we select a new category
def update_counts(change):
    count_widget.value = dataset.get_count(change['new'])
category_widget.observe(update_counts, names='value')

def save_snapshot(_, content, msg):
    if content['event'] == 'click':
        data = content['eventData']
        x = data['offsetX']
        y = data['offsetY']
        # save to disk
        dataset.save_entry(category_widget.value, camera.value, x, y)       
        # display saved snapshot
        snapshot = camera.value.copy()
        snapshot = cv2.circle(snapshot, (x, y), 8, (0, 255, 0), 3)
        snapshot_widget.value = bgr8_to_jpeg(snapshot)
        count_widget.value = dataset.get_count(category_widget.value)
camera_widget.on_msg(save_snapshot)
data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget, snapshot_widget]),
    dataset_widget,
    category_widget,
    count_widget
])
display(data_collection_widget)

#~ここからｺﾝﾄﾛｰﾗ~~~~~~~
import multiprocessing
import time
import pygame
import math
import numpy as np
from pygame.locals import *
from Adafruit_GPIO import I2C

# pygameの初期化
pygame.init()
# ジョイスティックの初期化
pygame.joystick.init()
try:
    # ジョイスティックインスタンスの生成
    joystick = pygame.joystick.Joystick(0)
    joystick.init()
    print("Joystick Name: " + joystick.get_name())
    print("Number of Button : " + str(joystick.get_numbuttons()))
    print("Number of Axis : " + str(joystick.get_numaxes()))
    print("Number of Hats : " + str(joystick.get_numhats()))
except pygame.error:
    print('ジョイスティックが接続されていません')

def expocurve(val):
    if val >0:
        return (math.exp(val)-1)/(math.e-1)
    else:
        return (math.exp(val*-1)-1)/(math.e-1)*-1
         
# F710の操作設定
# スティック
# 左スティック#０：左右、１：上下
# 右スティック#３：左右、４：上下
axis_steer = 0
axis_accel = 4
# ボタン
# 8: 'back', 9: 'start', 10: 'Logitech',
# 0: 'A', 1: 'B', 2: 'X', 3: 'Y',
# 4: 'LB',7: 'R1',8: 'right_stick_press',
button_A = 0
button_B = 1
button_Y = 3
button_X = 2
button_S = 7
button_R1 = 5
hat_rl = 0
hat_ud = 1

##スティックの指数関数変換
mode ="exp"

dirpath= "./road_following_A"

def save_snapshot_js2v(steering, throttle):
        x = steering *1 #マシン毎要調整
        y = throttle *-1 #マシン毎要調整
        x = int(camera.width * (x / 2.0 + 0.5))
        y = int(camera.height * (y / 2.0 + 0.5))  

        # save to disk
        dataset.save_entry(category_widget.value, camera.value, x, y)
        
        # display saved snapshot
        min, mid, max = int(224*0.2), int(224*0.5), int(224*0.8) #出力画像値の設定、マシン毎
        snapshot = camera.value.copy()
        snapshot = cv2.circle(snapshot, (x, y), 8, (0, 255, 0), 3)
        snapshot = cv2.line(snapshot, (mid, min), (mid, max), (255, 255, 255), thickness=1, lineType=cv2.LINE_4)
        snapshot = cv2.line(snapshot, (min, mid), (max, mid), (255, 255, 255), thickness=1, lineType=cv2.LINE_4)
        snapshot = cv2.rectangle(snapshot, (min, min), (max, max), (0, 255, 0)) #マシン毎要調整 52,171, (112)
        snapshot = cv2.putText(snapshot, (str(x)+":"+str(y)), (x-50, y-20), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0, 255, 0), 1, 8)
        snapshot_widget.value = bgr8_to_jpeg(snapshot)
        count_widget.value = dataset.get_count(category_widget.value)
        
def save_snapshot_js2v_noimage(steering, throttle,recording):
        while recording.value:
            x = steering.value *1 #マシン毎要調整
            y = throttle.value *-1 #マシン毎要調整
            x = int(camera.width * (x / 2.0 + 0.5))
            y = int(camera.height * (y / 2.0 + 0.5))  
            # save to disk
            #dataset.save_entry_norefresh(path, camera.value, x, y)
            dataset.save_entry(category_widget.value, camera.value, x, y)
            
            time.sleep(0.2)
            print("x,y: ",x,y)
            print("data recorded")
            #count_widget.value = dataset.get_count(category_widget.value)
        
AI = False
running = True
recording = multiprocessing.Value('b', False) 
#False
steering = multiprocessing.Value('f', 0) # Value型の共有オブジェクト
throttle = multiprocessing.Value('f', 0) # Value型の共有オブジェクト

while running:
    for e in pygame.event.get():
        A =joystick.get_button(button_A)
        B =joystick.get_button(button_B)
        Y =joystick.get_button(button_Y)
        X =joystick.get_button(button_X)
        S =joystick.get_button(button_S)
        R1 =joystick.get_button(button_R1)
        Hud = joystick.get_hat(0)[hat_ud]
        if Hud != 0:
            car.throttle_gain = round(car.throttle_gain+(Hud*0.05),2)
            print("Current ThGain:",car.throttle_gain)
        if X:
            car.steering = expocurve(round(joystick.get_axis(axis_steer),2) )
            car.throttle = 0.3 *car.throttle_gain
            """
            camera.running = not(camera.running)
            if camera.running:
                print("Start camera/AI running...")
            else:
                print("Stop camera/AI running...")
            """
        elif mode == "exp":
            car.steering = expocurve(round(joystick.get_axis(axis_steer),2) )
            car.throttle = expocurve(round(joystick.get_axis(axis_accel),2) *-1 )

        else:
            car.steering = round(joystick.get_axis(axis_steer),2) 
            car.throttle = round(joystick.get_axis(axis_accel),2) *-1 

        if A : 
            #camera.running = False
            print("Breaking!")
            car.steering = 0
            car.throttle = -0.5
            time.sleep(0.05)            
            car.throttle = 0
            time.sleep(0.02)            
            car.throttle = -0.5
            time.sleep(0.02)            
            car.throttle = 0
            if recording.value:
                recording.value = not recording.value
                p1.terminate()
                #p1.join()
                print("Stop") 
                time.sleep(0.2)

        if recording.value :
            steering.value=car.steering
            throttle.value=car.throttle
        if Y:
            recording.value = not recording.value
            print("recording:",recording)
            if recording.value:
                p1 = multiprocessing.Process(target=save_snapshot_js2v_noimage,args=(steering,throttle,recording,))
                p1.start()
                print("Start recording data!")
                time.sleep(0.2)
            else:
                p1.terminate()
                #p1.join()
                print("Stop") 
                time.sleep(0.2)
        if B:  
            save_snapshot_js2v(car.steering,car.throttle)
            print("Get running data!") 

        #f R1:  
        #   save_snapshot_js2v_noimage(car.steering,car.throttle)
        #   print("Get running data!") 

        if S:
            running = False
            print("Stop controlling...")



In [ ]:
from multiprocessing import Process, Queue
import os
import time
import random


# Queueにデータを書き込む
def write(q):
    print('Process to write: {}'.format(os.getpid()))
    for value in ['A', 'B', 'C']:
        print('Put {} to queue...'.format(value))
        q.put(value)
        time.sleep(random.random())


# Queueからデータを読み取り
def read(q):
    print('Process to read: {}'.format(os.getpid()))
    while True:
        value = q.get(True)
        print('Get {} from queue.'.format(value))


# 親プロセスがQueueを作って、子プロセスに渡す
q = Queue()
#pw = Process(target=write, args=(q,))
pr = Process(target=read, args=(q,))
# pwを起動し、書き込み開始
pw.start()
# prを起動し、読み取り開始
pr.start()
# pwが終了するのを待つ
pw.join()
#time.sleep(3)
# prは無限ループなので、強制終了
pr.terminate()


# カメラの終了処理(必須)

次のNotebookに進む際に必ずカメラの終了処理を実行してください。